# Let's play: Defining and training different models

Try different (simplified) models for classification and test performance

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import libraries and some constants

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm
import pandas as pd
import uproot as ur
import atlas_mpl_style as ampl
ampl.use_atlas_style()

In [3]:
# my worspace and directories

path_prefix = '/home/dportill/LCStudies/'
plotpath = path_prefix+'classifier/Plots/'
modelpath = path_prefix+'classifier/Models/'

In [4]:
# import our resolution utilities

import sys
sys.path.append(path_prefix)
from  util import resolution_util as ru
from  util import plot_util as pu
from  util import ml_util as mu

In [5]:
# Load data v7 with images from pubnote

inputpath = '/fast_scratch/atlas_images/v7/'
rootfiles = ["pi0", "piplus", "piminus"]

#setupPionData(inputpath, rootfiles, branches = []) on ml_util with defaultBranches from ClusterTree
trees, pdata = mu.setupPionData(inputpath, rootfiles)

In [6]:
# pdata -> Dictionary of dataframes. "pi0", "piplus", "piminus" are separated inside of the dictionary

np0 = len(pdata['pi0'])
npp = len(pdata['piplus'])
npm = len(pdata['piminus'])

print("Number of pi0 events: {}".format(np0))
print("Number of pi+ events: {}".format(npp))
print("Number of pi- events: {}".format(npm))
print("Total: {}".format(np0+npp+npm))

Number of pi0 events: 263891
Number of pi+ events: 435967
Number of pi- events: 434627
Total: 1134485


In [7]:
pcells = {
    ifile : {
        layer : mu.setupCells(itree, layer, flatten = False)
        for layer in mu.cell_meta
    }
    for ifile, itree in trees.items()
}

In [8]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow import keras as keras

In [9]:
import tensorflow as tf

In [10]:
## let's just take one gpu:
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [11]:
# create train/validation/test subsets containing 70%/10%/20%
# of events from each type of pion event
# merge pi0 and pi+ events
training_classes = ['pi0','piplus']
pdata_merged, pcells_merged, plabels = mu.createTrainingDatasets(training_classes, pdata, pcells)


In [12]:
pcellsE23_EMB2G_channels = mu.setupChannelImages(mu.rescaleImages(pcells_merged, (16, 16), layers=['EMB2', 'EMB3']),last=True)

In [13]:
pcellsE1_EMB1G_channels = mu.setupChannelImages(mu.rescaleImages(pcells_merged, (128, 4), layers=['EMB1']),last=True)

In [14]:
pcellsT123_T1G_channels = mu.setupChannelImages(mu.rescaleImages(pcells_merged, (4, 4), layers=['TileBar0', 'TileBar1', 'TileBar2']),last=True)

* CNN with EMB 1,2,3

In [16]:
def merged_model_EMB(droprate=.5):
    # EMB1 image (convolutional)
    input1 = Input(shape=(128,4,1), name='emb1_input')
    x1 = Convolution2D(32, (3, 3), padding='same', name='emb1_conv2d_1')(input1)
    x1 = Activation('relu')(x1)
    # x1 = Dropout(droprate)(x1)
    x1 = Convolution2D(32, (3, 3), padding='same', name='emb1_conv2d_2')(x1)
    x1 = Activation('relu')(x1)
    x1 = MaxPool2D(pool_size=(2, 1), padding='same', name='emb1_maxpool_3')(x1)
    x1 = Convolution2D(64, (3, 3), padding='same', name='emb1_conv2d_3')(x1)
    x1 = Activation('relu')(x1)
    #x1 = Dropout(droprate)(x1)
    x1 = Convolution2D(64, (3, 3), padding='same', name='emb1_conv2d_4')(x1)
    x1 = Activation('relu')(x1)
    x1 = MaxPool2D(pool_size=(2, 1), padding='same', name='emb1_maxpool_5')(x1)
    x1 = Convolution2D(128, (2, 2), padding='same', name='emb1_conv2d_6')(x1)
    x1 = Activation('relu')(x1)
    x1 = Convolution2D(128, (2, 2), padding='same', name='emb1_conv2d_7')(x1)
    x1 = Activation('relu')(x1)
    x1 = MaxPool2D(pool_size=(2, 1), padding='same', name='emb1_maxpool_8')(x1)
    x1 = Dropout(droprate, name='emb1_dropout_4')(x1)
    x1 = Flatten(name='emb1_flatten_9')(x1)
    x1 = Dense(128, activation='relu', name='emb1_dense_9')(x1)

    # EMB23 image (convolutional)
    input2 = Input(shape=(16,16,2), name='emb23_input')
    x2 = Convolution2D(32, (1, 1), padding='same', name='emb23_conv1d_1')(input2)
    x2 = Activation('relu')(x2)
    # x2 = Dropout(droprate)(x2)
    x2 = Convolution2D(64, (2, 2), padding='same', name='emb23_conv2d_2')(x2)
    # x2 = Dropout(droprate)(x2)
    x2 = MaxPool2D(pool_size=(2, 2), padding='same', name='emb23_maxpool_3')(x2)
    x2 = Convolution2D(128, (2, 2), padding='same', name='emb23_conv2d_4')(x2)
    x2 = Activation('relu')(x2)
    # x2 = Dropout(droprate)(x2)
    x2 = Convolution2D(128, (2, 2), padding='same', name='emb23_conv2d_5')(x2)
    x2 = Activation('relu')(x2)
    x2 = MaxPool2D(pool_size=(2, 2), padding='same', name='emb23_maxpool_6')(x2)
    x2 = Dropout(droprate, name='emb23_dropout_4')(x2)
    x2 = Flatten(name='emb23_flatten_7')(x2)
    x2 = Dense(128, activation='relu', name='emb23_dense_8')(x2)


    # concatenate outputs from the two networks above
    x = concatenate([x1, x2], name='concatenate') 
    x = Dropout(droprate, name='concate_dropout_5')(x)
    x = Dense(64, name='concated_dense_1')(x)    
    x = Activation('relu')(x)
    x = Dropout(droprate, name='dense_dropout_6')(x)

    # final output
    output = Dense(2, activation='softmax', name='dense_output')(x)
    # output = 5*tf.math.tanh(x)   # 0 to +5 range

    model = Model(inputs = [input1, input2], outputs = [output])
    
    # compile model
    # model = multi_gpu_model(model, gpus=4)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [17]:
model_p1_d02_EMB = merged_model_EMB(0.2)

In [19]:
##### Testing 
history_p1_d02_EMB = model_p1_d02_EMB.fit([pcellsE1_EMB1G_channels[pdata_merged.train], 
                                pcellsE23_EMB2G_channels[pdata_merged.train]],
                                plabels[pdata_merged.train],
                                validation_data=([pcellsE1_EMB1G_channels[pdata_merged.val], 
                                                    pcellsE23_EMB2G_channels[pdata_merged.val]], 
                                                    plabels[pdata_merged.val]),
                                epochs = 100,
                                batch_size = 128,
                                verbose=2
                                # use_multiprocessing=False
                                )

Epoch 1/100
3828/3828 - 39s - loss: 0.2221 - accuracy: 0.9144 - val_loss: 0.2101 - val_accuracy: 0.9183
Epoch 2/100
3828/3828 - 39s - loss: 0.2165 - accuracy: 0.9168 - val_loss: 0.2138 - val_accuracy: 0.9183
Epoch 3/100
3828/3828 - 39s - loss: 0.2126 - accuracy: 0.9182 - val_loss: 0.2043 - val_accuracy: 0.9215
Epoch 4/100
3828/3828 - 39s - loss: 0.2095 - accuracy: 0.9197 - val_loss: 0.2038 - val_accuracy: 0.9237
Epoch 5/100
3828/3828 - 39s - loss: 0.2040 - accuracy: 0.9232 - val_loss: 0.1976 - val_accuracy: 0.9257
Epoch 6/100
3828/3828 - 39s - loss: 0.1982 - accuracy: 0.9260 - val_loss: 0.1877 - val_accuracy: 0.9299
Epoch 7/100
3828/3828 - 39s - loss: 0.1926 - accuracy: 0.9287 - val_loss: 0.1835 - val_accuracy: 0.9306
Epoch 8/100
3828/3828 - 38s - loss: 0.1870 - accuracy: 0.9316 - val_loss: 0.1776 - val_accuracy: 0.9358
Epoch 9/100
3828/3828 - 39s - loss: 0.1838 - accuracy: 0.9330 - val_loss: 0.1797 - val_accuracy: 0.9353
Epoch 10/100
3828/3828 - 39s - loss: 0.1816 - accuracy: 0.9339 -

In [20]:
model_p1_d02_EMB.save(modelpath+'model_p1_d02_EMB.h5')

* CNN with EMB 1

In [21]:
def merged_model_EMB1(droprate=.5):
    # EMB1 image (convolutional)
    input1 = Input(shape=(128,4,1), name='emb1_input')
    x1 = Convolution2D(32, (3, 3), padding='same', name='emb1_conv2d_1')(input1)
    x1 = Activation('relu')(x1)
    # x1 = Dropout(droprate)(x1)
    x1 = Convolution2D(32, (3, 3), padding='same', name='emb1_conv2d_2')(x1)
    x1 = Activation('relu')(x1)
    x1 = MaxPool2D(pool_size=(2, 1), padding='same', name='emb1_maxpool_3')(x1)
    x1 = Convolution2D(64, (3, 3), padding='same', name='emb1_conv2d_3')(x1)
    x1 = Activation('relu')(x1)
    #x1 = Dropout(droprate)(x1)
    x1 = Convolution2D(64, (3, 3), padding='same', name='emb1_conv2d_4')(x1)
    x1 = Activation('relu')(x1)
    x1 = MaxPool2D(pool_size=(2, 1), padding='same', name='emb1_maxpool_5')(x1)
    x1 = Convolution2D(128, (2, 2), padding='same', name='emb1_conv2d_6')(x1)
    x1 = Activation('relu')(x1)
    x1 = Convolution2D(128, (2, 2), padding='same', name='emb1_conv2d_7')(x1)
    x1 = Activation('relu')(x1)
    x1 = MaxPool2D(pool_size=(2, 1), padding='same', name='emb1_maxpool_8')(x1)
    x1 = Dropout(droprate, name='emb1_dropout_4')(x1)
    x1 = Flatten(name='emb1_flatten_9')(x1)
    x1 = Dense(128, activation='relu', name='emb1_dense_9')(x1)

    ## Here was were we concatenated but now is just one input
    x1 = Dense(64, name='concated_dense_1')(x1)    
    x1 = Activation('relu')(x1)
    x1 = Dropout(droprate, name='dense_dropout_6')(x1)

    # final output
    output = Dense(2, activation='softmax', name='dense_output')(x1)
    # output = 5*tf.math.tanh(x)   # 0 to +5 range

    model = Model(inputs = [input1], outputs = [output])
    
    # compile model
    # model = multi_gpu_model(model, gpus=4)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [22]:
model_p1_d02_EMB1 = merged_model_EMB1(0.2)

In [24]:
history_p1_d02_EMB1 = model_p1_d02_EMB1.fit([pcellsE1_EMB1G_channels[pdata_merged.train]],
                                plabels[pdata_merged.train],
                                validation_data=([pcellsE1_EMB1G_channels[pdata_merged.val]], 
                                                    plabels[pdata_merged.val]),
                                epochs = 100,
                                batch_size = 128,
                                verbose=2
                                # use_multiprocessing=False
                                )

Epoch 1/100
3828/3828 - 29s - loss: 0.2484 - accuracy: 0.9064 - val_loss: 0.2413 - val_accuracy: 0.9082
Epoch 2/100
3828/3828 - 28s - loss: 0.2405 - accuracy: 0.9095 - val_loss: 0.2313 - val_accuracy: 0.9123
Epoch 3/100
3828/3828 - 29s - loss: 0.2356 - accuracy: 0.9112 - val_loss: 0.2366 - val_accuracy: 0.9099
Epoch 4/100
3828/3828 - 29s - loss: 0.2326 - accuracy: 0.9122 - val_loss: 0.2245 - val_accuracy: 0.9151
Epoch 5/100
3828/3828 - 29s - loss: 0.2286 - accuracy: 0.9139 - val_loss: 0.2239 - val_accuracy: 0.9147
Epoch 6/100
3828/3828 - 29s - loss: 0.2265 - accuracy: 0.9151 - val_loss: 0.2219 - val_accuracy: 0.9151
Epoch 7/100
3828/3828 - 29s - loss: 0.2248 - accuracy: 0.9151 - val_loss: 0.2270 - val_accuracy: 0.9150
Epoch 8/100
3828/3828 - 29s - loss: 0.2228 - accuracy: 0.9162 - val_loss: 0.2182 - val_accuracy: 0.9169
Epoch 9/100
3828/3828 - 29s - loss: 0.2217 - accuracy: 0.9169 - val_loss: 0.2150 - val_accuracy: 0.9189
Epoch 10/100
3828/3828 - 29s - loss: 0.2205 - accuracy: 0.9173 -

In [26]:
model_p1_d02_EMB1.save(modelpath+'model_p1_d02_EMB1.h5')

* CNN with EMB 2,3

In [28]:
def merged_model_EMB23(droprate=.5):

    # EMB23 image (convolutional)
    input2 = Input(shape=(16,16,2), name='emb23_input')
    x2 = Convolution2D(32, (1, 1), padding='same', name='emb23_conv1d_1')(input2)
    x2 = Activation('relu')(x2)
    # x2 = Dropout(droprate)(x2)
    x2 = Convolution2D(64, (2, 2), padding='same', name='emb23_conv2d_2')(x2)
    # x2 = Dropout(droprate)(x2)
    x2 = MaxPool2D(pool_size=(2, 2), padding='same', name='emb23_maxpool_3')(x2)
    x2 = Convolution2D(128, (2, 2), padding='same', name='emb23_conv2d_4')(x2)
    x2 = Activation('relu')(x2)
    # x2 = Dropout(droprate)(x2)
    x2 = Convolution2D(128, (2, 2), padding='same', name='emb23_conv2d_5')(x2)
    x2 = Activation('relu')(x2)
    x2 = MaxPool2D(pool_size=(2, 2), padding='same', name='emb23_maxpool_6')(x2)
    x2 = Dropout(droprate, name='emb23_dropout_4')(x2)
    x2 = Flatten(name='emb23_flatten_7')(x2)
    x2 = Dense(128, activation='relu', name='emb23_dense_8')(x2)


    ## Here was were we concatenated but now is just one input
    x2 = Dense(64, name='concated_dense_1')(x2)    
    x2 = Activation('relu')(x2)
    x2 = Dropout(droprate, name='dense_dropout_6')(x2)


    # final output
    output = Dense(2, activation='softmax', name='dense_output')(x2)
    # output = 5*tf.math.tanh(x)   # 0 to +5 range

    model = Model(inputs = [input2], outputs = [output])
    
    # compile model
    # model = multi_gpu_model(model, gpus=4)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model_p1_d02_EMB23.save(modelpath+'model_p1_d02_EMB23.h5')

* CNN with EMB 1,2,3

In [16]:
def merged_model_T123(droprate=.5):

  # tiles image (convolutional)
    input3 = Input(shape=(4,4,3), name='tiles_input')
    x3 = Convolution2D(32, (1, 1), padding='same', name='tiles_conv1d_1')(input3)
    x3 = Activation('relu')(x3)
    # x3 = Dropout(droprate)(x3)
    x3 = Convolution2D(64, (2, 2), padding='same', name='tiles_conv2d_2')(x3)
    x3 = Activation('relu')(x3)
    # x3 = Dropout(droprate)(x3)
    x3 = Convolution2D(128, (2, 2), padding='same', name='tiles_conv2d_3')(x3)
    x3 = Activation('relu')(x3)
    x3 = MaxPool2D(pool_size=(2, 2), padding='same', name='tiles_maxpool_4')(x3)
    x3 = Dropout(droprate, name='tiles_dropout_4')(x3)
    x3 = Flatten(name='tiles_flatten_5')(x3)
    x3 = Dense(128, activation='relu', name='tiles_dense_6')(x3)

    ## Here was were we concatenated but now is just one input
    x3 = Dense(64, name='concated_dense_1')(x3)    
    x3 = Activation('relu')(x3)
    x3 = Dropout(droprate, name='dense_dropout_6')(x3)


    # final output
    output = Dense(2, activation='softmax', name='dense_output')(x3)
    # output = 5*tf.math.tanh(x)   # 0 to +5 range

    model = Model(inputs = [input3], outputs = [output])
    
    # compile model
    # model = multi_gpu_model(model, gpus=4)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [19]:
model_p1_d02_T123 = merged_model_T123(0.2)

In [20]:
history_p1_d02_T123 = model_p1_d02_T123.fit([pcellsT123_T1G_channels[pdata_merged.train]],
                                plabels[pdata_merged.train],
                                validation_data=([pcellsT123_T1G_channels[pdata_merged.val]], 
                                                    plabels[pdata_merged.val]),
                                epochs = 100,
                                batch_size = 128,
                                verbose=2
                                # use_multiprocessing=False
                                )

Epoch 1/100
3828/3828 - 18s - loss: 0.4797 - accuracy: 0.7134 - val_loss: 0.4737 - val_accuracy: 0.7173
Epoch 2/100
3828/3828 - 17s - loss: 0.4640 - accuracy: 0.7171 - val_loss: 0.4396 - val_accuracy: 0.7184
Epoch 3/100
3828/3828 - 17s - loss: 0.4393 - accuracy: 0.7172 - val_loss: 0.4393 - val_accuracy: 0.7193
Epoch 4/100
3828/3828 - 17s - loss: 0.4384 - accuracy: 0.7182 - val_loss: 0.4328 - val_accuracy: 0.7192
Epoch 5/100
3828/3828 - 17s - loss: 0.4348 - accuracy: 0.7181 - val_loss: 0.4309 - val_accuracy: 0.7203
Epoch 6/100
3828/3828 - 17s - loss: 0.4336 - accuracy: 0.7193 - val_loss: 0.4275 - val_accuracy: 0.7202
Epoch 7/100
3828/3828 - 17s - loss: 0.4321 - accuracy: 0.7195 - val_loss: 0.4293 - val_accuracy: 0.7199
Epoch 8/100
3828/3828 - 17s - loss: 0.4311 - accuracy: 0.7193 - val_loss: 0.4284 - val_accuracy: 0.7184
Epoch 9/100
3828/3828 - 17s - loss: 0.4313 - accuracy: 0.7193 - val_loss: 0.4307 - val_accuracy: 0.7207
Epoch 10/100
3828/3828 - 17s - loss: 0.4305 - accuracy: 0.7196 -

In [22]:
model_p1_d02_T123.save(modelpath+'model_p1_d02_T123.h5')

In [23]:
def merged_model_2Conv2D(droprate=.5):
    # EMB1 image (convolutional)
    input1 = Input(shape=(128,4,1), name='emb1_input')
    x1 = Convolution2D(32, (3, 3), padding='same', name='emb1_conv2d_1')(input1)
    x1 = Activation('relu')(x1)
    # x1 = Dropout(droprate)(x1)
    x1 = Convolution2D(32, (3, 3), padding='same', name='emb1_conv2d_2')(x1)
    x1 = Activation('relu')(x1)
    x1 = MaxPool2D(pool_size=(2, 1), padding='same', name='emb1_maxpool_3')(x1)
    #x1 = Convolution2D(64, (3, 3), padding='same', name='emb1_conv2d_3')(x1)
    #x1 = Activation('relu')(x1)
    ##x1 = Dropout(droprate)(x1)
    #x1 = Convolution2D(64, (3, 3), padding='same', name='emb1_conv2d_4')(x1)
    #x1 = Activation('relu')(x1)
    #x1 = MaxPool2D(pool_size=(2, 1), padding='same', name='emb1_maxpool_5')(x1)
    #x1 = Convolution2D(128, (2, 2), padding='same', name='emb1_conv2d_6')(x1)
    #x1 = Activation('relu')(x1)
    #x1 = Convolution2D(128, (2, 2), padding='same', name='emb1_conv2d_7')(x1)
    #x1 = Activation('relu')(x1)
    #x1 = MaxPool2D(pool_size=(2, 1), padding='same', name='emb1_maxpool_8')(x1)
    
    x1 = Dropout(droprate, name='emb1_dropout_4')(x1)
    x1 = Flatten(name='emb1_flatten_9')(x1)
    x1 = Dense(128, activation='relu', name='emb1_dense_9')(x1)

    # EMB23 image (convolutional)
    input2 = Input(shape=(16,16,2), name='emb23_input')
    x2 = Convolution2D(32, (1, 1), padding='same', name='emb23_conv1d_1')(input2)
    x2 = Activation('relu')(x2)
    # x2 = Dropout(droprate)(x2)
    x2 = Convolution2D(64, (2, 2), padding='same', name='emb23_conv2d_2')(x2)
    ##### Missing activation 
    # x2 = Dropout(droprate)(x2)
    x2 = MaxPool2D(pool_size=(2, 2), padding='same', name='emb23_maxpool_3')(x2)
    #x2 = Convolution2D(128, (2, 2), padding='same', name='emb23_conv2d_4')(x2)
    #x2 = Activation('relu')(x2)
    ## x2 = Dropout(droprate)(x2)
    #x2 = Convolution2D(128, (2, 2), padding='same', name='emb23_conv2d_5')(x2)
    #x2 = Activation('relu')(x2)
    #x2 = MaxPool2D(pool_size=(2, 2), padding='same', name='emb23_maxpool_6')(x2)
    
    x2 = Dropout(droprate, name='emb23_dropout_4')(x2)
    x2 = Flatten(name='emb23_flatten_7')(x2)
    x2 = Dense(128, activation='relu', name='emb23_dense_8')(x2)

    # tiles image (convolutional)
    input3 = Input(shape=(4,4,3), name='tiles_input')
    x3 = Convolution2D(32, (1, 1), padding='same', name='tiles_conv1d_1')(input3)
    x3 = Activation('relu')(x3)
    # x3 = Dropout(droprate)(x3)
    x3 = Convolution2D(64, (2, 2), padding='same', name='tiles_conv2d_2')(x3)
    x3 = Activation('relu')(x3)
    # x3 = Dropout(droprate)(x3)
    #x3 = Convolution2D(128, (2, 2), padding='same', name='tiles_conv2d_3')(x3)
    #x3 = Activation('relu')(x3)
    x3 = MaxPool2D(pool_size=(2, 2), padding='same', name='tiles_maxpool_4')(x3)
    
    x3 = Dropout(droprate, name='tiles_dropout_4')(x3)
    x3 = Flatten(name='tiles_flatten_5')(x3)
    x3 = Dense(128, activation='relu', name='tiles_dense_6')(x3)

    # concatenate outputs from the two networks above
    x = concatenate([x1, x2, x3], name='concatenate') 
    x = Dropout(droprate, name='concate_dropout_5')(x)
    x = Dense(64, name='concated_dense_1')(x)    
    x = Activation('relu')(x)
    x = Dropout(droprate, name='dense_dropout_6')(x)

    # final output
    output = Dense(2, activation='softmax', name='dense_output')(x)
    # output = 5*tf.math.tanh(x)   # 0 to +5 range

    model = Model(inputs = [input1, input2, input3], outputs = [output])
    
    # compile model
    # model = multi_gpu_model(model, gpus=4)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [25]:
model_p1_d02_2Conv2D = merged_model_2Conv2D(0.2)

In [28]:
history_p1_d02_2Conv2D = model_p1_d02_2Conv2D.fit([pcellsE1_EMB1G_channels[pdata_merged.train], 
                                pcellsE23_EMB2G_channels[pdata_merged.train], 
                                pcellsT123_T1G_channels[pdata_merged.train]],
                                plabels[pdata_merged.train],
                                validation_data=([pcellsE1_EMB1G_channels[pdata_merged.val], 
                                                    pcellsE23_EMB2G_channels[pdata_merged.val], 
                                                    pcellsT123_T1G_channels[pdata_merged.val]], 
                                                    plabels[pdata_merged.val]),
                                epochs = 100,
                                batch_size = 128,
                                verbose=2
                                # use_multiprocessing=False
                                )

Epoch 1/100
3828/3828 - 26s - loss: 0.1995 - accuracy: 0.9264 - val_loss: 0.1843 - val_accuracy: 0.9319
Epoch 2/100
3828/3828 - 25s - loss: 0.1947 - accuracy: 0.9282 - val_loss: 0.1863 - val_accuracy: 0.9307
Epoch 3/100
3828/3828 - 25s - loss: 0.1913 - accuracy: 0.9300 - val_loss: 0.1824 - val_accuracy: 0.9332
Epoch 4/100
3828/3828 - 25s - loss: 0.1888 - accuracy: 0.9307 - val_loss: 0.1804 - val_accuracy: 0.9350
Epoch 5/100
3828/3828 - 25s - loss: 0.1871 - accuracy: 0.9313 - val_loss: 0.1838 - val_accuracy: 0.9328
Epoch 6/100
3828/3828 - 25s - loss: 0.1855 - accuracy: 0.9321 - val_loss: 0.1830 - val_accuracy: 0.9325
Epoch 7/100
3828/3828 - 25s - loss: 0.1841 - accuracy: 0.9328 - val_loss: 0.1770 - val_accuracy: 0.9356
Epoch 8/100
3828/3828 - 24s - loss: 0.1822 - accuracy: 0.9334 - val_loss: 0.1722 - val_accuracy: 0.9374
Epoch 9/100
3828/3828 - 25s - loss: 0.1807 - accuracy: 0.9340 - val_loss: 0.1714 - val_accuracy: 0.9381
Epoch 10/100
3828/3828 - 25s - loss: 0.1796 - accuracy: 0.9347 -

In [29]:
model_p1_d02_2Conv2D.save(modelpath+'model_p1_d02_2Conv2D.h5')

In [32]:
def merged_model_1Conv2D(droprate=.5):
    # EMB1 image (convolutional)
    input1 = Input(shape=(128,4,1), name='emb1_input')
    x1 = Convolution2D(32, (3, 3), padding='same', name='emb1_conv2d_1')(input1)
    #x1 = Activation('relu')(x1)
    # x1 = Dropout(droprate)(x1)
    #x1 = Convolution2D(32, (3, 3), padding='same', name='emb1_conv2d_2')(x1)
    x1 = Activation('relu')(x1)
    x1 = MaxPool2D(pool_size=(2, 1), padding='same', name='emb1_maxpool_3')(x1)
    #x1 = Convolution2D(64, (3, 3), padding='same', name='emb1_conv2d_3')(x1)
    #x1 = Activation('relu')(x1)
    ##x1 = Dropout(droprate)(x1)
    #x1 = Convolution2D(64, (3, 3), padding='same', name='emb1_conv2d_4')(x1)
    #x1 = Activation('relu')(x1)
    #x1 = MaxPool2D(pool_size=(2, 1), padding='same', name='emb1_maxpool_5')(x1)
    #x1 = Convolution2D(128, (2, 2), padding='same', name='emb1_conv2d_6')(x1)
    #x1 = Activation('relu')(x1)
    #x1 = Convolution2D(128, (2, 2), padding='same', name='emb1_conv2d_7')(x1)
    #x1 = Activation('relu')(x1)
    #x1 = MaxPool2D(pool_size=(2, 1), padding='same', name='emb1_maxpool_8')(x1)
    
    x1 = Dropout(droprate, name='emb1_dropout_4')(x1)
    x1 = Flatten(name='emb1_flatten_9')(x1)
    x1 = Dense(128, activation='relu', name='emb1_dense_9')(x1)

    # EMB23 image (convolutional)
    input2 = Input(shape=(16,16,2), name='emb23_input')
    x2 = Convolution2D(32, (1, 1), padding='same', name='emb23_conv1d_1')(input2)
    x2 = Activation('relu')(x2)
    # x2 = Dropout(droprate)(x2)
    #x2 = Convolution2D(64, (2, 2), padding='same', name='emb23_conv2d_2')(x2)
    ##### Missing activation 
    # x2 = Dropout(droprate)(x2)
    x2 = MaxPool2D(pool_size=(2, 2), padding='same', name='emb23_maxpool_3')(x2)
    #x2 = Convolution2D(128, (2, 2), padding='same', name='emb23_conv2d_4')(x2)
    #x2 = Activation('relu')(x2)
    ## x2 = Dropout(droprate)(x2)
    #x2 = Convolution2D(128, (2, 2), padding='same', name='emb23_conv2d_5')(x2)
    #x2 = Activation('relu')(x2)
    #x2 = MaxPool2D(pool_size=(2, 2), padding='same', name='emb23_maxpool_6')(x2)
    
    x2 = Dropout(droprate, name='emb23_dropout_4')(x2)
    x2 = Flatten(name='emb23_flatten_7')(x2)
    x2 = Dense(128, activation='relu', name='emb23_dense_8')(x2)

    # tiles image (convolutional)
    input3 = Input(shape=(4,4,3), name='tiles_input')
    x3 = Convolution2D(32, (1, 1), padding='same', name='tiles_conv1d_1')(input3)
    #x3 = Activation('relu')(x3)
    # x3 = Dropout(droprate)(x3)
    #x3 = Convolution2D(64, (2, 2), padding='same', name='tiles_conv2d_2')(x3)
    x3 = Activation('relu')(x3)
    # x3 = Dropout(droprate)(x3)
    #x3 = Convolution2D(128, (2, 2), padding='same', name='tiles_conv2d_3')(x3)
    #x3 = Activation('relu')(x3)
    x3 = MaxPool2D(pool_size=(2, 2), padding='same', name='tiles_maxpool_4')(x3)
    
    x3 = Dropout(droprate, name='tiles_dropout_4')(x3)
    x3 = Flatten(name='tiles_flatten_5')(x3)
    x3 = Dense(128, activation='relu', name='tiles_dense_6')(x3)

    # concatenate outputs from the two networks above
    x = concatenate([x1, x2, x3], name='concatenate') 
    x = Dropout(droprate, name='concate_dropout_5')(x)
    x = Dense(64, name='concated_dense_1')(x)    
    x = Activation('relu')(x)
    x = Dropout(droprate, name='dense_dropout_6')(x)

    # final output
    output = Dense(2, activation='softmax', name='dense_output')(x)
    # output = 5*tf.math.tanh(x)   # 0 to +5 range

    model = Model(inputs = [input1, input2, input3], outputs = [output])
    
    # compile model
    # model = multi_gpu_model(model, gpus=4)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [33]:
model_p1_d02_1Conv2D = merged_model_1Conv2D(0.2)

In [38]:
history_p1_d02_1Conv2D = model_p1_d02_1Conv2D.fit([pcellsE1_EMB1G_channels[pdata_merged.train], 
                                pcellsE23_EMB2G_channels[pdata_merged.train], 
                                pcellsT123_T1G_channels[pdata_merged.train]],
                                plabels[pdata_merged.train],
                                validation_data=([pcellsE1_EMB1G_channels[pdata_merged.val], 
                                                    pcellsE23_EMB2G_channels[pdata_merged.val], 
                                                    pcellsT123_T1G_channels[pdata_merged.val]], 
                                                    plabels[pdata_merged.val]),
                                epochs = 100,
                                batch_size = 128,
                                verbose=2
                                # use_multiprocessing=False
                                )

Epoch 1/100
3828/3828 - 20s - loss: 0.1808 - accuracy: 0.9341 - val_loss: 0.1771 - val_accuracy: 0.9354
Epoch 2/100
3828/3828 - 20s - loss: 0.1804 - accuracy: 0.9339 - val_loss: 0.1691 - val_accuracy: 0.9385
Epoch 3/100
3828/3828 - 20s - loss: 0.1806 - accuracy: 0.9341 - val_loss: 0.1697 - val_accuracy: 0.9377
Epoch 4/100
3828/3828 - 20s - loss: 0.1801 - accuracy: 0.9342 - val_loss: 0.1702 - val_accuracy: 0.9370
Epoch 5/100
3828/3828 - 20s - loss: 0.1804 - accuracy: 0.9339 - val_loss: 0.1759 - val_accuracy: 0.9361
Epoch 6/100
3828/3828 - 20s - loss: 0.1802 - accuracy: 0.9341 - val_loss: 0.1693 - val_accuracy: 0.9380
Epoch 7/100
3828/3828 - 20s - loss: 0.1799 - accuracy: 0.9343 - val_loss: 0.1700 - val_accuracy: 0.9379
Epoch 8/100
3828/3828 - 20s - loss: 0.1804 - accuracy: 0.9342 - val_loss: 0.1693 - val_accuracy: 0.9382
Epoch 9/100
3828/3828 - 20s - loss: 0.1797 - accuracy: 0.9344 - val_loss: 0.1716 - val_accuracy: 0.9370
Epoch 10/100
3828/3828 - 20s - loss: 0.1799 - accuracy: 0.9341 -

In [39]:
model_p1_d02_1Conv2D.save(modelpath+'model_p1_d02_1Conv2D_v2.h5')